<a href="https://colab.research.google.com/github/genarioazevedoufape/estudos_data_science/blob/main/BLEU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import math

# Baixa recursos necessários do NLTK
nltk.download('punkt')

def normalizar_texto(texto):
    """Normaliza o texto convertendo para minúsculas e tokenizando."""
    return word_tokenize(texto.lower())

def contar_ngramas(tokens, n):
    """Gera n-gramas e suas contagens a partir de uma lista de tokens."""
    return Counter(tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1))

def precisao_modificada(candidato, referencias, n):
    """Calcula a precisão modificada de n-gramas para um candidato contra referências."""
    ngramas_candidato = contar_ngramas(candidato, n)
    max_ngramas_referencia = Counter()

    # Encontra a contagem máxima de cada n-grama entre todas as referências
    for ref in referencias:
        ngramas_ref = contar_ngramas(ref, n)
        for ngrama in ngramas_ref:
            max_ngramas_referencia[ngrama] = max(max_ngramas_referencia[ngrama], ngramas_ref[ngrama])

    # Limita (clip) as contagens do candidato ao máximo das referências
    contagens_clipadas = sum(min(ngramas_candidato[ngrama], max_ngramas_referencia[ngrama]) for ngrama in ngramas_candidato)
    contagens_totais = sum(ngramas_candidato.values())

    # Evita divisão por zero
    return contagens_clipadas / contagens_totais if contagens_totais > 0 else 0.0

def penalidade_brevidade(candidato, referencias):
    """Calcula a penalidade de brevidade com base no comprimento do candidato e da referência mais próxima."""
    c = len(candidato)
    # Encontra o comprimento da referência mais próximo do candidato
    r = min((len(ref) for ref in referencias), key=lambda x: abs(x - c))

    if c >= r:
        return 1.0
    return math.exp(1 - r / c)

def escore_bleu(candidato, referencias, max_n=4, pesos=None):
    """Calcula o escore BLEU para um candidato contra múltiplas referências."""
    if pesos is None:
        pesos = [1.0 / max_n] * max_n

    # Normaliza os textos
    candidato = normalizar_texto(candidato)
    referencias = [normalizar_texto(ref) for ref in referencias]

    # Calcula as precisões modificadas para n-gramas
    precisoes = []
    for n in range(1, max_n + 1):
        p_n = precisao_modificada(candidato, referencias, n)
        precisoes.append(p_n)

    # Aplica suavização para evitar precisões zero (adiciona pequeno epsilon)
    precisoes = [p if p > 0 else 1e-16 for p in precisoes]

    # Média geométrica das precisões
    soma_log = sum(w * math.log(p) for w, p in zip(pesos, precisoes))
    media_geometrica = math.exp(soma_log)

    # Penalidade de brevidade
    bp = penalidade_brevidade(candidato, referencias)

    return bp * media_geometrica

# Casos de teste para demonstrar o cálculo do escore BLEU
def executar_testes_bleu():
    """Executa casos de teste para demonstrar o comportamento da métrica BLEU."""
    casos_teste = [
        {
            "nome": "Correspondência Perfeita",
            "candidato": "O gato está no tapete",
            "referencias": ["O gato está no tapete", "Existe um gato no tapete"],
            "descricao": "O candidato corresponde exatamente a uma referência."
        },
        {
            "nome": "Problema com Sinônimo",
            "candidato": "O cão está no tapete",
            "referencias": ["O gato está no tapete", "Existe um gato no tapete"],
            "descricao": "O candidato usa 'cão' em vez de 'gato' (problema com sinônimo)."
        },
        {
            "nome": "Diferença na Ordem das Palavras",
            "candidato": "No tapete está o gato",
            "referencias": ["O gato está no tapete"],
            "descricao": "Mesmas palavras, mas ordem diferente afeta bigramas e superiores."
        },
        {
            "nome": "Tradução Truncada",
            "candidato": "O gato",
            "referencias": ["O gato está no tapete", "Existe um gato no tapete"],
            "descricao": "Candidato curto ativa a penalidade de brevidade."
        },
        {
            "nome": "Candidato Repetitivo",
            "candidato": "O o o",
            "referencias": ["O gato está no tapete"],
            "descricao": "Palavras repetitivas testam o clipping na precisão modificada."
        }
    ]

    for teste in casos_teste:
        escore = escore_bleu(teste["candidato"], teste["referencias"])
        print(f"\nCaso de Teste: {teste['nome']}")
        print(f"Descrição: {teste['descricao']}")
        print(f"Candidato: {teste['candidato']}")
        print(f"Referências: {teste['referencias']}")
        print(f"Escore BLEU: {escore:.4f}")

        # Mostra as precisões individuais de n-gramas
        tokens_candidato = normalizar_texto(teste["candidato"])
        tokens_referencias = [normalizar_texto(ref) for ref in teste["referencias"]]
        for n in range(1, 5):
            p_n = precisao_modificada(tokens_candidato, tokens_referencias, n)
            print(f"Precisão (n={n}): {p_n:.4f}")

# Executa os testes quando o script é rodado
if __name__ == "__main__":
    executar_testes_bleu()


Caso de Teste: Correspondência Perfeita
Descrição: O candidato corresponde exatamente a uma referência.
Candidato: O gato está no tapete
Referências: ['O gato está no tapete', 'Existe um gato no tapete']
Escore BLEU: 1.0000
Precisão (n=1): 1.0000
Precisão (n=2): 1.0000
Precisão (n=3): 1.0000
Precisão (n=4): 1.0000

Caso de Teste: Problema com Sinônimo
Descrição: O candidato usa 'cão' em vez de 'gato' (problema com sinônimo).
Candidato: O cão está no tapete
Referências: ['O gato está no tapete', 'Existe um gato no tapete']
Escore BLEU: 0.0001
Precisão (n=1): 0.8000
Precisão (n=2): 0.5000
Precisão (n=3): 0.3333
Precisão (n=4): 0.0000

Caso de Teste: Diferença na Ordem das Palavras
Descrição: Mesmas palavras, mas ordem diferente afeta bigramas e superiores.
Candidato: No tapete está o gato
Referências: ['O gato está no tapete']
Escore BLEU: 0.0000
Precisão (n=1): 1.0000
Precisão (n=2): 0.5000
Precisão (n=3): 0.0000
Precisão (n=4): 0.0000

Caso de Teste: Tradução Truncada
Descrição: Candi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
import math
import collections
from fractions import Fraction
from typing import List, Dict, Tuple

def simple_tokenizer(text: str) -> List[str]:
    """Tokenizador simples que divide o texto em palavras e converte para minúsculas"""
    return text.lower().split()

def count_ngrams(tokens: List[str], n: int) -> Dict[Tuple[str, ...], int]:
    """Conta a ocorrência de n-gramas em uma lista de tokens"""
    ngrams = collections.defaultdict(int)
    for i in range(len(tokens) - n + 1):
        ngram = tuple(tokens[i:i+n])
        ngrams[ngram] += 1
    return ngrams

def modified_precision(candidate: List[str], references: List[List[str]], n: int) -> float:
    """Calcula a precisão modificada para n-gramas"""
    candidate_ngrams = count_ngrams(candidate, n)
    if not candidate_ngrams:
        return 0.0

    max_ref_ngrams = collections.defaultdict(int)
    for ref in references:
        ref_ngrams = count_ngrams(ref, n)
        for ngram, count in ref_ngrams.items():
            max_ref_ngrams[ngram] = max(max_ref_ngrams[ngram], count)

    clipped_counts = 0
    total_counts = 0

    for ngram, count in candidate_ngrams.items():
        clipped_counts += min(count, max_ref_ngrams.get(ngram, 0))
        total_counts += count

    return clipped_counts / total_counts if total_counts > 0 else 0.0

def brevity_penalty(candidate: List[str], references: List[List[str]]) -> float:
    """Calcula a penalidade de brevidade"""
    c = len(candidate)
    ref_lens = [len(ref) for ref in references]
    r = min(ref_lens, key=lambda x: abs(x - c))

    if c > r:
        return 1.0
    else:
        return math.exp(1 - r / c) if c > 0 else 0.0

def bleu_score(candidate: str, references: List[str], weights: List[float] = None, tokenizer=None) -> float:
    """Calcula o score BLEU para uma candidata e múltiplas referências"""
    if weights is None:
        weights = [0.25, 0.25, 0.25, 0.25]  # Padrão para BLEU-4

    if tokenizer is None:
        tokenizer = simple_tokenizer

    # Tokeniza as sentenças
    candidate_tokens = tokenizer(candidate)
    reference_tokens = [tokenizer(ref) for ref in references]

    # Calcula as precisões modificadas para cada n-grama
    precisions = []
    for i in range(len(weights)):
        p = modified_precision(candidate_tokens, reference_tokens, i+1)
        precisions.append(p)

    # Calcula a penalidade de brevidade
    bp = brevity_penalty(candidate_tokens, reference_tokens)

    # Calcula a média geométrica ponderada das precisões
    sum_log_p = 0.0
    for w, p in zip(weights, precisions):
        if p > 0:
            sum_log_p += w * math.log(p)
        else:
            # Se qualquer precisão for zero, o BLEU será zero
            return 0.0

    # Calcula o score final
    bleu = bp * math.exp(sum_log_p)
    return bleu

def corpus_bleu(candidates: List[str], references: List[List[str]], weights: List[float] = None, tokenizer=None) -> float:
    """Calcula o BLEU para um corpus inteiro (múltiplas sentenças)"""
    if weights is None:
        weights = [0.25, 0.25, 0.25, 0.25]

    if tokenizer is None:
        tokenizer = simple_tokenizer

    # Calcula BLEU para cada par candidato-referência
    individual_scores = []
    for cand, refs in zip(candidates, references):
        score = bleu_score(cand, refs, weights, tokenizer)
        individual_scores.append(score)

    # Retorna a média dos scores individuais
    return sum(individual_scores) / len(individual_scores) if individual_scores else 0.0

In [25]:
# Exemplo 1: Tradução perfeita
ref1 = "o gato está no tapete"
cand1 = "o gato está no tapete"
print(f"Exemplo 1 - BLEU: {bleu_score(cand1, [ref1])}")  # Deve ser 1.0

# Exemplo 2: Tradução com ordem invertida
ref2 = "o gato persegue o rato"
cand2 = "o rato persegue o gato"
print(f"Exemplo 2 - BLEU: {bleu_score(cand2, [ref2])}")  # Deve ser 0.0 (devido aos bigramas)

# Exemplo 3: Tradução muito curta
ref3 = "o gato está sobre o tapete"
cand3 = "o gato"
print(f"Exemplo 3 - BLEU: {bleu_score(cand3, [ref3])}")  # Deve ser baixo devido à penalidade de brevidade

# Exemplo 4: Tradução com repetição
ref4 = "o gato está no tapete"
cand4 = "o o o o o"
print(f"Exemplo 4 - BLEU: {bleu_score(cand4, [ref4])}")  # Deve ser 0.0

# Exemplo 5: Múltiplas referências
refs5 = ["o gato está no tapete", "há um gato no tapete"]
cand5 = "o gato está sobre o tapete"
print(f"Exemplo 5 - BLEU: {bleu_score(cand5, refs5)}")  # Deve ser entre 0 e 1

# Exemplo 6: Corpus completo
corpus_candidates = [
    "o gato está no tapete",
    "o cachorro late",
    "o pássaro canta"
]
corpus_references = [
    ["o gato está no tapete", "há um gato no tapete"],
    ["o cão late", "o cachorro faz au au"],
    ["a ave canta", "o pássaro está cantando"]
]
print(f"Exemplo 6 - Corpus BLEU: {corpus_bleu(corpus_candidates, corpus_references)}")

Exemplo 1 - BLEU: 1.0
Exemplo 2 - BLEU: 0.0
Exemplo 3 - BLEU: 0.0
Exemplo 4 - BLEU: 0.0
Exemplo 5 - BLEU: 0.0
Exemplo 6 - Corpus BLEU: 0.3333333333333333


In [26]:
!pip install nltk

In [27]:
import nltk
# Ensure the 'punkt_tab' resource is downloaded
nltk.download('punkt_tab')
nltk.download('punkt') # Keep the existing download as well
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu as nltk_corpus_bleu
from nltk.tokenize import word_tokenize

# Função para tokenizar com NLTK
def nltk_tokenizer(text):
    return word_tokenize(text.lower())

# Testando com NLTK
print("\nComparação com NLTK:")
print(f"Nossa implementação: {bleu_score(cand1, [ref1], tokenizer=nltk_tokenizer)}")
print(f"NLTK: {sentence_bleu([nltk_tokenizer(ref1)], nltk_tokenizer(cand1))}")

# Testando corpus BLEU
print("\nCorpus BLEU - Nossa implementação:", corpus_bleu(corpus_candidates, corpus_references, tokenizer=nltk_tokenizer))
print("Corpus BLEU - NLTK:", nltk_corpus_bleu([[nltk_tokenizer(r) for r in refs] for refs in corpus_references],
                                             [nltk_tokenizer(c) for c in corpus_candidates]))


Comparação com NLTK:
Nossa implementação: 1.0
NLTK: 1.0

Corpus BLEU - Nossa implementação: 0.3333333333333333
Corpus BLEU - NLTK: 0.6887246539984299


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
# Teste com sinônimos
ref_syn = "o cachorro está no tapete"
cand_syn = "o cão está no tapete"
print(f"\nTeste com sinônimos - BLEU: {bleu_score(cand_syn, [ref_syn])}")  # Será baixo pois BLEU não reconhece sinônimos

# Teste com sentença mais longa que a referência
ref_long = "o gato está no tapete"
cand_long = "o gato preto está deitado no tapete azul"
print(f"Teste com sentença longa - BLEU: {bleu_score(cand_long, [ref_long])}")  # Não há penalidade por ser mais longa

# Teste com smoothing (para evitar zeros)
def smoothed_bleu(candidate, references, weights=None, tokenizer=None): # Add tokenizer parameter
    if weights is None:
        weights = [0.25, 0.25, 0.25, 0.25]

    if tokenizer is None: # Add default tokenizer
        tokenizer = simple_tokenizer

    # Aplica um pequeno smoothing para evitar log(0)
    precisions = []
    for i in range(len(weights)):
        p = modified_precision(tokenizer(candidate), [tokenizer(ref) for ref in references], i+1)
        precisions.append(p + 1e-10)  # Smoothing

    bp = brevity_penalty(tokenizer(candidate), [tokenizer(ref) for ref in references])
    sum_log_p = sum(w * math.log(p) for w, p in zip(weights, precisions))
    return bp * math.exp(sum_log_p)

print(f"Teste com smoothing - BLEU: {smoothed_bleu(cand2, [ref2])}")  # Agora não será zero absoluto


Teste com sinônimos - BLEU: 0.0
Teste com sentença longa - BLEU: 0.0
Teste com smoothing - BLEU: 9.306048591563838e-06
